Login to your azure account az login - 

Ensure you select the Python 3.10 PyTorch and Tensorflow Kernel

In [ ]:
# Install required packages for authentication
!pip install python-dotenv azure-identity azure-ai-ml -U

In [ ]:
pip install azure-ai-inference

In [ ]:
# Set up Azure authentication using service principal or interactive login
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, AzureCliCredential
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
# Load environment variables
print("Loading environment variables from .env file...")
load_dotenv()

# Get Azure ML credentials
subscription_id = os.getenv('AZUREML_SUBSCRIPTION_ID')
resource_group = os.getenv('AZUREML_RESOURCE_GROUP')
workspace = os.getenv('AZUREML_WS_NAME')

# Print values for debugging (without showing actual secrets)
print(f"Subscription ID loaded: {'Yes' if subscription_id else 'No'}")
print(f"Resource Group loaded: {'Yes' if resource_group else 'No'}")
print(f"Workspace Name loaded: {'Yes' if workspace else 'No'}")

# Try different authentication methods
try:
    print("Attempting authentication with AzureCliCredential...")
    ml_client = MLClient(
        AzureCliCredential(),
        subscription_id,
        resource_group,
        workspace
    )
    print("Authentication successful with AzureCliCredential!")
except Exception as e:
    print(f"AzureCliCredential failed with error: {str(e)}")
    print("\nTrying interactive browser authentication...")
    try:
        # Fall back to interactive authentication
        credential = InteractiveBrowserCredential()
        ml_client = MLClient(
            credential,
            subscription_id,
            resource_group,
            workspace
        )
        print("Authentication successful with InteractiveBrowserCredential!")
    except Exception as e2:
        print(f"InteractiveBrowserCredential failed with error: {str(e2)}")
        print("\nPlease run 'az login' in a terminal and try again.")
        print("If running in a Jupyter environment, you might need to:")
        print("1. Run 'az login' in a terminal outside Jupyter")
        print("2. Update your .env file with correct credentials")

In [ ]:
pip  install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 -U

In [ ]:
! pip install olive-ai[auto-opt] -U

In [ ]:
pip install onnxruntime-genai==0.7.0 --pre

In [ ]:
! pip install onnxruntime

In [ ]:
! pip install bitsandbytes

In [ ]:
! pip install transformers==4.49.0 -U

In [ ]:
! pip install azure-ai-ml -U  

In [ ]:
! pip install marshmallow==3.23.2 -U   

In [ ]:
! pip install tf-keras

In [ ]:
! pip install numpy==1.23.5 -U

In [ ]:
! pip install peft

In [ ]:
from azure.identity import AzureCliCredential
credential = AzureCliCredential()

In [ ]:
! olive finetune \
    --method lora \
    --model_name_or_path  azureml://registries/azureml/models/Phi-4-mini-instruct/versions/1 \
    --trust_remote_code \
    --data_name json \
    --data_files ./data/train_data.jsonl \
    --text_template "<|user|>{Question}<|end|><|assistant|>{Answer}<|end|>" \
    --max_steps 100 \
    --output_path models/phi-4-mini/ft \
    --target_modules "q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj" \
    --log_level 1

In [ ]:
! pip install onnxruntime-genai==0.7.0 --pre

In [ ]:
! olive auto-opt \
    --model_name_or_path  azureml://registries/azureml/models/Phi-4-mini-instruct/versions/1 \
    --adapter_path models/phi-4-mini/ft/adapter \
    --device cpu \
    --provider CPUExecutionProvider \
    --use_model_builder \
    --precision int4 \
    --output_path models/phi-4-mini/onnx \
    --log_level 1